In [ ]:
# First we import the module and start up the Matlab interpreter

import os
import numpy as np
from pace_neutrons import Matlab
m = Matlab()

swp = m.swpref()
swp.usemex = 1

### Fitting with Horace and SpinW

In this practical we will be doing a fit of the bcc-Iron dataset using both Horace and SpinW.

Start by looking back at the `day2_horace3_solution.ipynb` notebook where we did a fit of the iron dataset using an analytic Python function - copy the code to make a series of Q-cuts integrating in 20meV chunks between 80 and 160meV and then the code to fit this using an analytic function

In [ ]:
???

Now create a SpinW model of a body-centred cubic lattice (space group `I m -3 m` - you canuse the `'spgr', 'I m -3 m'` keyword in the `genlattice` method of SpinW) with an Fe$^{3+}$ (`'MFe3'`) ion at $(0,0,0)$.

Add one nearest neighbour exchange interaction labelled `'J1'` and set the magnetic structure as ferromagnetic with moments in the $c$ direction (note there are two Fe atoms in the unit cell).

In [ ]:
???

Now to fit we have to pass a Matlab function handle for the SpinW fit function to Horace we have to use a round-about way to do this (better implementation in future!):

```
m.assignin('base', 'fe', fe)
spinw_sqw = m.evalin('base', '@fe.horace_sqw')
```

Then create a `multifit_sqw` object with this handle as the fit function, and set the parameters as follows:

```
mfobj = mfobj.set_fun(spinw_sqw)
mfobj = mfobj.set_pin((pars_in_spinw, 'mat', ('J1',), 'resfun', 'sho', 'formfact', True, 'hermit', False))
```

This uses a simple-harmonic oscillator function as the energy broadening term - this takes two parameters $\Gamma$ and $T$ the temperature - we want to fix the temperature.

Finally there is the scalling factor (amplitude `Amp`) which is the last parameter so the parameter vector is:

```
pars_in = [J, Gamma, Temperature, Amp]
```

but


In [ ]:
m.assignin('base', 'fe', fe)
spinw_sqw = m.evalin('base', '@fe.horace_sqw')

Temperature = 10   # Measurement done at 10K

pars_in_spinw = [J, Gamma, Temperature, Amp]

mfobj = m.multifit_sqw(my_cuts)
mfobj = mfobj.set_fun(spinw_sqw)
mfobj = mfobj.set_pin((pars_in_spinw, 'mat', ('J1',), 'resfun', 'sho', 'formfact', True, 'hermit', False))
mfobj = mfobj.set_free([1, 1, 0, 1])
mfobj = mfobj.set_bfun(linear_bg, [0.3, 0])
m.tic()
wfit = mfobj.simulate()
t1 = m.toc()

print(f'Time to run fit with SpinW function is {t0} s.')

%matlab_fig -h 800 -w 600 -r 100

# Plots the cuts using a vectorised operation, offsetting each cut by 0.5 units in y
m.acolor(['black', 'red', 'blue', 'green', 'yellow'])
m.amark(['+', 'o', '*', '.', 'x', 's'], 6)
my_cut_offset = my_cuts + np.arange(0, 2.1, 0.5)
my_fit_offset = wfit + np.arange(0, 2.1, 0.5)
m.dp(my_cut_offset)
m.pl(my_fit_offset)